Instituto Politécnico Nacional

Centro de Investigación en Computación

Minería de datos I

Ejercicios del capítulo 10. Cluster Analysis, Basic Concepts and Methods.

Ejercicios 1 al 6 y 8.

Presenta: Ana Maritza Bello

---

10.1 Describa brevemente y dé ejemplos de cada uno de los siguientes enfoques de agrupación: métodos de partición, métodos jerárquicos, métodos basados en densidad y métodos basados en cuadrículas.

10.2 Supongamos que la tarea de minería de datos es agrupar puntos (donde _(x, y)_ representa la ubicación) en tres grupos, donde los puntos son:

**A1 (2, 10), A2 (2, 5), A3 (8, 4), B1 (5, 8), B2 (7, 5), B3 (6, 4), C1 (1, 2), C2 (4, 9)**.

La función de distancia es la distancia euclidiana. Supongamos que inicialmente asignamos _A1, B1_ y _C1_ como el centro de cada grupo, respectivamente. Utilice el algoritmo _k-means_ para mostrar solo
1.  Los tres centros temáticos después de la primera ronda de ejecución.
1.  Los tres últimos grupos.

10.3 Utilice un ejemplo para mostrar por qué el algoritmo _k-means_ puede no encontrar el óptimo global, es decir, optimizar la variación dentro del grupo.

10.4 Para el algoritmo _k-means_, es interesante observar que al elegir
cuidadosamente los centros de agrupamiento iniciales, podemos no solo acelerar
la convergencia del algoritmo, sino también garantizar la calidad del
agrupamiento final. El algoritmo **k-means++** es una variante de _k-means_, que
elige los centros iniciales de la siguiente manera. Primero, selecciona un
centro uniformemente al azar de los objetos del conjunto de datos. De forma
iterativa, para cada objeto p distinto del centro elegido, elige un objeto como
nuevo centro. Este objeto se elige al azar con probabilidad proporcional a
_dist(p)^2_, donde _dist(p)_ es la distancia desde _p_ al centro más cercano que ya
ha sido elegido. La iteración continúa hasta que se seleccionan _k_ centros.

Explique por qué este método no solo acelerará la convergencia del algoritmo _k-means_, sino que también garantizará la calidad de los resultados finales de agrupación.

10.5 Proporcionar el pseudocódigo del paso de reasignación de objetos del algoritmo PAM.

10.6 Tanto los algoritmos de _k-means_ como de _k-medoides_ pueden realizar una
agrupación eficaz.

(a) Ilustre la fortaleza y debilidad de _k-means_ en comparación con _k-medoides_.
(b) Ilustre las fortalezas y debilidades de estos esquemas en comparación con un esquema de agrupamiento jerárquico (por ejemplo, AGNES).

10.8 Demuestre que en DBSCAN, para un valor fijo de _MinPts_ y dos umbrales de
vecindad, $\epsilon_1 < \epsilon_2$, un grupo _C_ con respecto a $\epsilon_1$ y
_MinPts_ debe ser un subconjunto de un grupo _C'_ con respecto a $\epsilon_2$ y _MinPts_.